# Adding Fully Connected Layers and Model Summary


## Step 1: Setting Up PyTorch Environment


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Step 2: Defining Fully Connected Layers in PyTorch


In [ ]:
class CNNWithFC(nn.Module):
    def __init__(self, input_channels=1, image_height=28, image_width=28, num_classes=10):
        super(CNNWithFC, self).__init__()
        self.input_channels = input_channels
        self.image_height = image_height
        self.image_width = image_width
        self.num_classes = num_classes

        # Convolutional layers
        # Conv2d(in_channels, out_channels, kernel_size, padding)
        self.conv1 = nn.Conv2d(self.input_channels, 32, kernel_size=3, padding=1)
        # With kernel_size=3 and padding=1, spatial dimensions (height, width) are preserved.
        # Output shape: (batch_size, 32, image_height, image_width)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        # Spatial dimensions are preserved by this convolution as well.
        # Input shape to conv2 (after pooling): (batch_size, 32, image_height/2, image_width/2)
        # Output shape of conv2: (batch_size, 64, image_height/2, image_width/2)

        # Max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Each pooling operation halves the spatial dimensions.

        # Calculate the flattened size of the feature map after conv and pool layers
        # Initial dimensions: image_height, image_width
        # After conv1 (dims preserved), then pool1 (dims halved):
        # height_after_pool1 = image_height // 2
        # width_after_pool1 = image_width // 2
        # After conv2 (dims preserved), then pool2 (dims halved again):
        pooled_height = self.image_height // 2 // 2
        pooled_width = self.image_width // 2 // 2

        # The number of output channels from the last convolutional layer (conv2) is 64.
        self.fc1_in_features = 64 * pooled_height * pooled_width

        # Fully connected layers
        self.fc1 = nn.Linear(self.fc1_in_features, 128)
        self.fc2 = nn.Linear(128, self.num_classes) # Output layer: num_classes units for classification

    def forward(self, x):
        # Convolutional Block 1
        x = self.conv1(x)
        x = F.relu(x)    # Apply ReLU activation
        x = self.pool(x) # Apply Max Pooling

        # Convolutional Block 2
        x = self.conv2(x)
        x = F.relu(x)    # Apply ReLU activation
        x = self.pool(x) # Apply Max Pooling

        # Flatten the output from convolutional layers for the fully connected layers
        # x.size(0) is the batch size. -1 infers the remaining dimensions.
        x = x.view(x.size(0), -1)

        # Fully Connected Block
        x = F.relu(self.fc1(x)) # Apply ReLU activation
        x = self.fc2(x)         # Output raw logits (no activation for CrossEntropyLoss)
        return x

## Step 3: Visualizing the Model Summary

In [ ]:
# Ensure torchsummary is installed. If not, uncomment and run the next line in a code cell:
# !pip install torchsummary
from torchsummary import summary

# Define input parameters for the model, matching the example (e.g., MNIST dataset)
input_channels = 1    # Grayscale images
image_height = 28     # Image height of 28 pixels
image_width = 28      # Image width of 28 pixels
num_classes = 10      # Number of output classes (e.g., digits 0-9)

# Create an instance of the network with specified parameters
model = CNNWithFC(input_channels=input_channels,
                  image_height=image_height,
                  image_width=image_width,
                  num_classes=num_classes)

# Generate and print the model summary
# input_size for torchsummary is (channels, height, width)
summary(model, input_size=(input_channels, image_height, image_width))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
         MaxPool2d-2           [-1, 32, 14, 14]               0
            Conv2d-3           [-1, 64, 14, 14]          18,496
         MaxPool2d-4             [-1, 64, 7, 7]               0
            Linear-5                  [-1, 128]         401,536
            Linear-6                   [-1, 10]           1,290
Total params: 421,642
Trainable params: 421,642
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.36
Params size (MB): 1.61
Estimated Total Size (MB): 1.97
----------------------------------------------------------------
